In [6]:
#THIS IS FOR USING JUST ONE GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
import json
from tqdm import tqdm

Load dataset

In [8]:
with open("perfect_subset_phi.json", "r") as f:
    dataset = json.load(f)

print(f"Loaded {len(dataset)} examples.")

Loaded 200 examples.


-Load finetune model from Hugging Face

In [9]:

model = AutoModelForCausalLM.from_pretrained("karou1182001/fine-tuned-phi")
tokenizer = AutoTokenizer.from_pretrained("karou1182001/fine-tuned-phi")
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")
print("Model loaded from Hugging Face.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 28.27it/s]


Model loaded from Hugging Face.


Evaluate dataset

In [10]:
correct = 0
first_printed = False

for item in tqdm(dataset, desc="Evaluating..."):
    question = item["question"].strip()
    gold = str(item["gold_number"])

    # Zero-shot prompt 
    prompt = question + "\nA:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    pred_numbers = re.findall(r"\d+", decoded)
    prediction = pred_numbers[-1] if pred_numbers else None

    if not first_printed:
        print("\nFirst Example:")
        print(f"Question: {question}")
        print(f"Expected (gold): {gold}")
        print(f"Predicted: {prediction}")
        first_printed = True

    if prediction == gold:
        correct += 1

total = len(dataset)
accuracy = correct / total * 100

print(f"\nCorrect predictions: {correct}/{total}")
print(f"Accuracy on dataset 2: {accuracy:.2f}%")


Evaluating...:   0%|          | 1/200 [00:00<02:59,  1.11it/s]


First Example:
Question: There are some cats on the roof. Before the weaver begins assembling the garment, she carefully checks her inventory to avoid any surprises during production. A robe takes 10 bolts of blue fiber and half that much white fiber. How many bolts in total does it take? After finding the result, she notes it down to order supplies for next week.
Expected (gold): 15
Predicted: 40


Evaluating...: 100%|██████████| 200/200 [02:38<00:00,  1.26it/s]


Correct predictions: 13/200
Accuracy on dataset 2: 6.50%
